In [0]:
!pip install tensorflow==2.1.0
!pip install tensorflow-gpu
!pip install transformers
from google.colab import drive
drive.mount('/content/drive/')

     |████████████████████████████████| 421.8MB 34kB/s 
     |████████████████████████████████| 3.9MB 47.5MB/s 
     |████████████████████████████████| 450kB 42.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=2fdd47c280b5722fe09d715c9eb3eaf59fe99884504f4c873e84c6c7ddfef3ac
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installatio

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from keras.utils import to_categorical
import os
import six
from functools import partial
import numpy as np
import scipy as sp
from sklearn.metrics import f1_score
from transformers import *
import transformers
print(tf.__version__)

2.1.0


In [0]:
PATH ='/content/drive/My Drive/Colab Notebooks/data'
MAX_SEQUENCE_LENGTH = 150
input_categories = '微博中文内容'
output_categories = '情感倾向'
df_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/nCoV_100k_train.labled.csv')
df_train = df_train[df_train[output_categories].isin(['-1','0','1'])]
df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/nCov_10k_test.csv')
df_sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/submit_example.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

train shape = (99913, 7)
test shape = (10000, 6)


In [0]:
df_train.columns

Index(['微博id', '微博发布时间', '发布人账号', '微博中文内容', '微博图片', '微博视频', '情感倾向'], dtype='object')

In [0]:
def _convert_to_transformer_inputs(instance, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids, input_masks, input_segments = return_id(
        instance, 'longest_first', max_sequence_length)
    
    return [input_ids, input_masks, input_segments]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for instance in tqdm(df[columns]):
        
        ids, masks, segments = \
        _convert_to_transformer_inputs(str(instance), tokenizer, max_sequence_length)
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)
           ]

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

  0%|          | 256/99913 [00:00<00:38, 2556.17it/s]

100%|██████████| 10000/10000 [00:05<00:00, 1874.44it/s]


In [0]:
inputs

[array([[ 101, 1091, 1762, ...,    0,    0,    0],
        [ 101, 2458, 2399, ...,    0,    0,    0],
        [ 101, 6937, 3247, ...,    0,    0,    0],
        ...,
        [ 101,  108, 2544, ...,    0,    0,    0],
        [ 101,  523, 3173, ...,    0,    0,    0],
        [ 101,  794, 6073, ...,    0,    0,    0]], dtype=int32),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)]

In [0]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns].astype(int) + 1)
outputs = compute_output_arrays(df_train, output_categories)

In [0]:
def create_model():
    input_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)    
    input_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)    
    input_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32) 
    config = BertConfig.from_pretrained('bert-base-chinese')
    bert_model = TFBertModel.from_pretrained('bert-base-chinese',config=config)
    embedding = bert_model(input_id, attention_mask=input_mask, token_type_ids=input_atn)[0]   
    x = tf.keras.layers.GlobalAveragePooling1D()(embedding)    
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(3, activation='softmax',name='class_out')(x)
    model = tf.keras.models.Model(inputs=[input_id, input_mask, input_atn], outputs=x)
    return model

In [0]:
gkf = StratifiedKFold(n_splits=5,shuffle=True,random_state=0).split(X=df_train[input_categories].fillna('-1'), y=df_train[output_categories].fillna('-1'))
valid_preds = []
test_preds = []
for fold, (train_idx, valid_idx) in enumerate(gkf):
        train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
        train_outputs = to_categorical(outputs[train_idx])
        valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
        valid_outputs = to_categorical(outputs[valid_idx])
        K.clear_session()
        optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
        model = create_model()
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc', 'mae'])
        model.summary()
        model.fit(train_inputs, train_outputs, validation_data= [valid_inputs, valid_outputs], epochs=1,batch_size=64)
        valid_preds.append(model.predict(valid_inputs))
        test_preds.append(model.predict(test_inputs))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 150, 768), ( 102267648   input_1[0][0]                    
______________________________________________________________________________________________

In [0]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = [1.,1.,1.]
    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        X_p = np.argmax(X_p*coef,axis=1)
        y_t = np.argmax(y,axis=1)
        ll = f1_score(y_t, X_p,average='macro')
        print('f1 score: ',ll)
        return -ll
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if type(self.coef_) is list:
          initial_coef = self.coef_
        else:
          initial_coef = self.coef_['x']
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef,method='Nelder-Mead')
    def predict(self, X, coef):
        X_p = np.copy(X)
        X_p = np.argmax(X_p*coef,axis=1)
        return X_p
    def coefficients(self):
        return self.coef_['x']

In [0]:
len(valid_preds)

5

In [0]:
opr = OptimizedRounder()

In [0]:
gkf = StratifiedKFold(n_splits=5,shuffle=True,random_state=0).split(X=df_train[input_categories].fillna('-1'), y=df_train[output_categories].fillna('-1'))
for fold, (train_idx, valid_idx) in enumerate(gkf):
  print('flod: ',fold)
  valid_outputs = to_categorical(outputs[valid_idx])
  opr.fit(X=valid_preds[fold],y=valid_outputs)

flod:  0
f1 score:  0.7229792996734314
f1 score:  0.7236591941826706
f1 score:  0.720171353803464
f1 score:  0.7254160081231914
f1 score:  0.728543059257044
f1 score:  0.732195289360769
f1 score:  0.7310862046336348
f1 score:  0.7319496321039004
f1 score:  0.7362101285300638
f1 score:  0.7378396718366371
f1 score:  0.7391343911984917
f1 score:  0.7370411614890502
f1 score:  0.737626000384937
f1 score:  0.7317587397042998
f1 score:  0.7356894674945341
f1 score:  0.7357672395007363
f1 score:  0.7368900827181574
f1 score:  0.7383573319856732
f1 score:  0.7371830847561324
f1 score:  0.7380849736529592
f1 score:  0.7376615722808886
f1 score:  0.7376824992661705
f1 score:  0.7389393234665157
f1 score:  0.7377934894320429
f1 score:  0.7377910359448601
f1 score:  0.7375309635070981
f1 score:  0.7386685988603517
f1 score:  0.7376999237214062
f1 score:  0.7386428850621333
f1 score:  0.7375291315111799
f1 score:  0.739013410532666
f1 score:  0.7375367378905301
f1 score:  0.739013410532666
f1 scor

In [0]:
coef = opr.coef_['x']
sub = np.average(test_preds, axis=0)
sub = opr.predict(sub,coef)

In [0]:
df_sub['y'] = sub-1
df_sub.astype('int64').to_csv('test_sub.csv',index=False, encoding='utf-8')

In [0]:
target = np.argmax(valid_outputs,axis=1)
preds = np.argmax(valid_preds[4],axis=1)
tf.compat.v1.confusion_matrix(target,preds)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1723, 1488,  170],
       [ 629, 9265, 1629],
       [  44, 1052, 3982]], dtype=int32)>

In [0]:
!/opt/bin/nvidia-smi

Fri May  1 06:49:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  